In [1]:
import csv

import pandas as pd

In [2]:
!wget -q "https://raw.githubusercontent.com/yuremartins/Projeto-de-Rentabilidade-Pricing/main/dataset/car_rental_data.csv" -O car_rental_data.csv

In [3]:
with open('car_rental_data.csv', mode='r', encoding='utf-8') as arquivo_csv:
  data = list(csv.DictReader(arquivo_csv))

In [4]:
rental_df = pd.DataFrame(data)
rental_df.head()

,id_customer,id_vehicle,desc_vehicle,purchase_value,selling_value,segment,contract_duration
0,customer_33,vehicle_3,FIAT-NOVA STRADA ENDURANCE CS 1.4,81702,53620,engineering,8
1,customer_59,vehicle_8,GM-S10 CABINE SIMPLES,175555,123152,energy,29
2,customer_25,vehicle_2,VW-GOL TREND 1.6 FLEX 4P,34083,26254,other,14
3,customer_39,vehicle_7,HILUX CS DSL 4X4,262854,199660,agribusiness,16
4,customer_51,vehicle_9,MITSUBISHI-L200 TRITON GL 2.4 D 4X4 MT CD,201850,96047,mining,29


In [5]:
def calcular_taxa_depreciacao(valor_compra, valor_venda):
    valor_depreciacao = ((valor_compra - valor_venda) / valor_compra) * 100
    return round(valor_depreciacao, 2)

for index, row_deprec in rental_df.iterrows():
    valor_compra = float(row_deprec["purchase_value"])
    valor_venda = float(row_deprec["selling_value"])
    taxa_depreciacao = calcular_taxa_depreciacao(valor_compra, valor_venda)
    rental_df.at[index, "taxa_depreciacao"] = taxa_depreciacao

rental_df.head()

,id_customer,id_vehicle,desc_vehicle,purchase_value,selling_value,segment,contract_duration,taxa_depreciacao
0,customer_33,vehicle_3,FIAT-NOVA STRADA ENDURANCE CS 1.4,81702,53620,engineering,8,34.37
1,customer_59,vehicle_8,GM-S10 CABINE SIMPLES,175555,123152,energy,29,29.85
2,customer_25,vehicle_2,VW-GOL TREND 1.6 FLEX 4P,34083,26254,other,14,22.97
3,customer_39,vehicle_7,HILUX CS DSL 4X4,262854,199660,agribusiness,16,24.04
4,customer_51,vehicle_9,MITSUBISHI-L200 TRITON GL 2.4 D 4X4 MT CD,201850,96047,mining,29,52.42


In [6]:
def calcular_valor_locacao(valor_compra, valor_venda, taxa_lucro_desejada, prazo_locacao_desejado):
    # Cálculo do valor de locação necessário para atingir a taxa de lucro desejada
    valor_locacao_necessario = ((valor_compra + (valor_compra * taxa_lucro_desejada)) - valor_venda) / prazo_locacao_desejado

    # Cálculo do rental rate (porcentagem do valor de locação em relação ao valor de compra)
    rental_rate = (valor_locacao_necessario / valor_compra) * 100

    faturamento_total = (valor_locacao_necessario * prazo_locacao_desejado) + valor_venda

    return round(valor_locacao_necessario, 2), round(rental_rate, 2), round(faturamento_total, 2)

# Criar as colunas no DataFrame para armazenar os resultados
rental_df["valor_locacao_necessario"] = 0.0
rental_df["rental_rate"] = 0.0
rental_df["faturamento_total"] = 0.0

# Itera sobre as linhas do DataFrame e calcula o valor de locação para cada veículo
for index, row in rental_df.iterrows():
    valor_compra = float(row["purchase_value"])
    valor_venda = float(row["selling_value"])
    taxa_lucro = 0.10  # Taxa de lucro desejada
    prazo_locacao = int(row["contract_duration"])  # Prazo de locação desejado em meses

    valor_locacao_necessario, rental_rate, faturamento_total = calcular_valor_locacao(valor_compra, valor_venda, taxa_lucro, prazo_locacao)

    # Atribui os valores calculados às colunas correspondentes no DataFrame
    rental_df.at[index, "valor_locacao_necessario"] = valor_locacao_necessario
    rental_df.at[index, "rental_rate"] = rental_rate
    rental_df.at[index, "faturamento_total"] = faturamento_total

# Imprime o DataFrame atualizado
rental_df.head()


,id_customer,id_vehicle,desc_vehicle,purchase_value,selling_value,segment,contract_duration,taxa_depreciacao,valor_locacao_necessario,rental_rate,faturamento_total
0,customer_33,vehicle_3,FIAT-NOVA STRADA ENDURANCE CS 1.4,81702,53620,engineering,8,34.37,4531.52,5.55,89872.2
1,customer_59,vehicle_8,GM-S10 CABINE SIMPLES,175555,123152,energy,29,29.85,2412.36,1.37,193110.5
2,customer_25,vehicle_2,VW-GOL TREND 1.6 FLEX 4P,34083,26254,other,14,22.97,802.66,2.36,37491.3
3,customer_39,vehicle_7,HILUX CS DSL 4X4,262854,199660,agribusiness,16,24.04,5592.46,2.13,289139.4
4,customer_51,vehicle_9,MITSUBISHI-L200 TRITON GL 2.4 D 4X4 MT CD,201850,96047,mining,29,52.42,4344.41,2.15,222035.0


In [7]:
rental_df["contract_duration"] = pd.to_numeric(rental_df["contract_duration"], errors="coerce")

In [8]:
mean_rental_rate_dict = rental_df.groupby("segment")["rental_rate"].mean().to_dict()

statistical_analysis = rental_df.groupby("segment").agg({
    "taxa_depreciacao": "mean",
    "contract_duration": "mean",
    "rental_rate": "mean"
})

statistical_analysis["mean_rental_rate"] = statistical_analysis.index.map(mean_rental_rate_dict)
statistical_analysis["rental_rate_recomendado"] = statistical_analysis["mean_rental_rate"] * statistical_analysis["contract_duration"] / statistical_analysis["contract_duration"].mean()

statistical_analysis.rename(columns={
    "taxa_depreciacao": "mean_taxa_depreciacao",
    "contract_duration": "mean_contract_duration",
}, inplace=True)

statistical_analysis.head()

,mean_taxa_depreciacao,mean_contract_duration,rental_rate,mean_rental_rate,rental_rate_recomendado
segment,,,,,
agribusiness,25.205316,18.028947,2.039579,2.039579,1.993908
energy,32.610201,24.221106,1.937303,1.937303,2.544402
engineering,29.922557,10.242718,4.359450,4.359450,2.421258
mining,50.707197,25.224215,2.579507,2.579507,3.528162
other,24.864627,14.492537,2.878993,2.878993,2.262451
